In [32]:
import numpy as np
import pandas as pd
import re,math
from sklearn import preprocessing
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
#from bokeh.plotting import figure, show, output_file


data1 = pd.read_excel(r'pcbi.1006752.s001.xlsx', sheet_name="Validated targets",index_col='Unnamed: 0')
data2 = pd.read_excel(r'Label_synergy4000.xlsx')

In [33]:
temp1 = []
index_l = []
for index,row in data2.iterrows():
    ct = np.array(data1.loc[row["DrugA"],:]) + np.array(data1.loc[row["DrugB"],:])
    tp = []
    for i in ct:
        if i >= 1:
            tp.append(1)
        else:
            tp.append(0)
    temp1.append(tp)
    index_l.append((row["DrugA"],row["DrugB"]))

In [34]:
np.shape(index_l)

(583, 2)

In [35]:
def tanimoto(p,q):
    tep1=0
    tep2=0
    lenthp=int(len(p))
    for i in range(lenthp):
        a=p[i]        
        b=q[i]
        if (a==1)|(b==1):
            tep1=tep1+1
        if (a==1)&(b==1):
            tep2=tep2+1 
    c=float(tep2 / tep1)        
    return c

In [36]:
temp=[]  
for i in temp1:
    tmp=[]   
    a=np.array(i) #取出一组值
    for j in temp1:
        b=np.array(j)    #取出一组值
        tp=tanimoto(a,b)
        tmp.append(tp)
    temp.append(tmp)
Tanimoto=pd.DataFrame(temp,columns=index_l,index=index_l)

In [37]:
Tanimoto

,"(MK-5108, SORAFENIB)","(VINORELBINE, SUNITINIB)","(SUNITINIB, MK-8776)","(5-FU, DINACICLIB)","(SUNITINIB, MK-2206)","(PACLITAXEL, BEZ-235)","(MK-2206, DINACICLIB)","(MK-4827, MK-8776)","(VINORELBINE, DINACICLIB)","(MK-5108, MK-8776)",...,"(BORTEZOMIB, TOPOTECAN)","(BORTEZOMIB, SORAFENIB)","(GEMCITABINE, ERLOTINIB)","(CARBOPLATIN, MK-4827)","(METHOTREXATE, PD325901)","(CARBOPLATIN, MK-2206)","(5-FU, LAPATINIB)","(ETOPOSIDE, AZD1775)","(BORTEZOMIB, MK-8669)","(OXALIPLATIN, DINACICLIB)"
"(MK-5108, SORAFENIB)",1.000000,0.428571,0.428571,0.000000,0.375000,0.000000,0.000,0.000000,0.000000,0.083333,...,0.000000,0.769231,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.0,0.000000
"(VINORELBINE, SUNITINIB)",0.428571,1.000000,0.800000,0.000000,0.666667,0.000000,0.000,0.000000,0.076923,0.000000,...,0.000000,0.400000,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.0,0.000000
"(SUNITINIB, MK-8776)",0.428571,0.800000,1.000000,0.000000,0.666667,0.000000,0.000,0.090909,0.000000,0.100000,...,0.000000,0.400000,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.0,0.000000
"(5-FU, DINACICLIB)",0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.400,0.000000,0.500000,0.000000,...,0.090909,0.000000,0.181818,0.111111,0.0,0.100,0.333333,0.0,0.0,0.714286
"(SUNITINIB, MK-2206)",0.375000,0.666667,0.666667,0.000000,1.000000,0.000000,0.200,0.000000,0.000000,0.000000,...,0.000000,0.352941,0.000000,0.000000,0.0,0.250,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(CARBOPLATIN, MK-2206)",0.000000,0.000000,0.000000,0.100000,0.250000,0.000000,0.375,0.000000,0.000000,0.000000,...,0.125000,0.000000,0.111111,0.166667,0.0,1.000,0.125000,0.0,0.0,0.125000
"(5-FU, LAPATINIB)",0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,...,0.111111,0.000000,0.375000,0.142857,0.0,0.125,1.000000,0.0,0.0,0.111111
"(ETOPOSIDE, AZD1775)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,1.0,0.0,0.000000
"(BORTEZOMIB, MK-8669)",0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000,0.000000,0.000000,0.000000,...,0.333333,0.153846,0.000000,0.000000,0.0,0.000,0.000000,0.0,1.0,0.000000


In [38]:
ttpp = []
for i in list(Tanimoto.columns):
    ttpp.append(str(i)[1:-1])
data2 = pd.DataFrame(data2.iloc[:,2:].T.values, columns=list(ttpp))
columns = data2.columns

In [39]:
# 删去对应行后的字典
dict_lines = {}
count = 0
ct = 0
for i in Tanimoto.index:
    y = list(Tanimoto.iloc[:,ct])
    y.pop(count)
    count += 1
    dict_lines[columns[ct]] = y
    ct += 1

In [40]:
len(dict_lines)

583

In [41]:
data2

,"'MK-5108', 'SORAFENIB'","'VINORELBINE', 'SUNITINIB'","'SUNITINIB', 'MK-8776'","'5-FU', 'DINACICLIB'","'SUNITINIB', 'MK-2206'","'PACLITAXEL', 'BEZ-235'","'MK-2206', 'DINACICLIB'","'MK-4827', 'MK-8776'","'VINORELBINE', 'DINACICLIB'","'MK-5108', 'MK-8776'",...,"'BORTEZOMIB', 'TOPOTECAN'","'BORTEZOMIB', 'SORAFENIB'","'GEMCITABINE', 'ERLOTINIB'","'CARBOPLATIN', 'MK-4827'","'METHOTREXATE', 'PD325901'","'CARBOPLATIN', 'MK-2206'","'5-FU', 'LAPATINIB'","'ETOPOSIDE', 'AZD1775'","'BORTEZOMIB', 'MK-8669'","'OXALIPLATIN', 'DINACICLIB'"
0,-9.510,-13.200,26.400,4.33000,45.50,-5.010,9.970,76.20,-16.0000,-4.970,...,-12.20000,5.680,4.29,7.7300,-19.400,1.350,9.190,15.400,-6.3000,-3.460
1,2.600,-4.030,14.500,-8.16000,22.50,-3.860,1.690,21.60,-19.2000,11.100,...,-2.59000,-9.620,1.38,9.0600,-8.910,11.700,8.010,12.400,4.7900,-10.000
2,15.200,11.100,29.500,-5.42000,22.80,25.500,13.200,38.70,-123.0000,8.950,...,-52.80000,-1.510,-25.20,0.5210,-6.630,20.200,7.500,19.200,-23.3000,-12.000
3,6.220,10.500,17.500,-7.75000,18.00,4.910,7.660,-12.10,-4.4200,6.890,...,-0.00549,-0.520,6.67,-4.1700,11.500,9.170,-4.180,-5.010,-2.7000,-10.200
4,-16.400,-15.600,17.700,-14.30000,20.10,-15.300,19.200,22.40,-27.2000,-1.160,...,-3.90000,-32.200,-2.07,-0.9100,-0.722,46.600,-6.780,37.400,0.0856,-21.100
5,5.360,16.300,10.300,4.81000,22.70,21.200,7.130,13.50,17.4000,-48.400,...,-12.00000,-19.300,4.98,14.5000,2.630,0.586,3.210,8.590,-17.3000,1.130
6,-7.080,-1.720,0.830,0.31200,6.30,-30.800,-9.490,-12.00,-13.6000,16.800,...,-23.40000,-11.900,5.72,1.8200,19.200,-37.700,7.240,9.770,19.2000,-13.400
7,8.050,8.890,5.770,3.41000,24.90,4.260,14.700,-53.00,11.5000,3.900,...,-21.50000,-23.700,4.40,-11.8000,-11.400,-2.100,5.340,3.770,17.4000,0.261
8,12.400,0.686,19.300,2.92000,35.40,-6.130,4.010,17.40,-25.1000,8.670,...,-2.20000,-4.900,-3.28,15.0000,-28.200,34.100,9.690,10.600,1.5900,-5.080
9,5.790,9.680,8.400,-3.48000,23.40,27.100,6.520,21.80,-0.0348,-6.360,...,-5.68000,-8.800,3.60,1.5000,0.584,18.300,4.670,18.200,-11.6000,-6.190


In [42]:
# 分离x y数据构造字典
dict_columns = {}
for i in columns: 
    
    # b
    y = data2.loc[:,i]
    
    # A
    temp = []
    for j in columns:
        if j != i:
            temp.append(j)
    m = data2.loc[:, temp[0]]
    tp = [m]
    for j in temp[1:]:
        tp.append(data2.loc[:, j])
    x = pd.DataFrame(tp).T
    
    # 找每列x相关度较大的5列
    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_lines[i], index=tp)
    df.sort_values(by=0 , inplace=True, ascending=True)
    max_l = df.index[-5:]
    tp = []
    for j in max_l:
        tp.append(data2.loc[:, j])
    max_5x = pd.DataFrame(tp).T
    
    # 存储
    data = {
        "x": x,
        "y": y,
        "max_relevance": max_l,
        "max_5x": max_5x
    }
    dict_columns[i] = data

In [43]:
# 删去对应行后的字典
dict_high_lines = {}
count = 0
cct = 0
for i in Tanimoto.index:
    y = list(Tanimoto.iloc[:,cct])
    y.pop(count)
    count += 1
    dict_high_lines[str(i)[1:-1]] = y
    # 找每列x相关度较大的5行
    tp = []
    for j in columns:
        if str(i)[1:-1] != j:
            tp.append(j)
    df = pd.DataFrame(dict_high_lines[str(i)[1:-1]], index=tp)
    df.sort_values(by=0 , inplace=True, ascending=True)
    # print(df)
    max_l = df.index[-5:] #取最大5个
    tp = []
    for j in max_l:
        tp.append(Tanimoto.iloc[list(columns).index(j), :])
    max_5y = list(pd.DataFrame(tp).T.iloc[[list(columns).index(str(i)[1:-1])]].values)[0]
#     print(max_5y)
#     print(max_l)
#     break
    data = {
        "high_12": y,
        "max_relevance": max_l,
        "high_5": max_5y
    }
    dict_high_lines[str(i)[1:-1]] = data
    cct += 1

In [44]:
end_score = []
name_dict_high_lines = []
for elem in dict_high_lines:
    end_score.append(dict_high_lines[elem]['high_5'].mean())
    name_dict_high_lines.append(elem)
dict_high_lines_temp = {}
for score in end_score:
    if score >= 0.90:
        dict_high_lines_temp[name_dict_high_lines[end_score.index(score)]] = dict_high_lines[name_dict_high_lines[end_score.index(score)]]
print(len(dict_high_lines_temp))
dict_high_lines = dict_high_lines_temp.copy()

16


In [45]:
def dictW(alpha):
    # 运算构造参数W字典 
    dict_W = {}
    for elem in dict_high_lines:
        r = np.array(dict_high_lines[elem]["high_5"])
        W1= pow((1-r),2)
        W = np.exp(-W1/(2*pow(alpha,2)))
        dict_W[elem] = W
    return dict_W

#计算MSE
def dictr(dict_W):
# 残差字典 
    # r = max(|A×W - b|)
    dict_r = {}
    dict_r_list = {}
    for elem,w in zip(dict_columns, dict_W):
        A = np.array(dict_columns[elem]["max_5x"].values) # x----->max_5x
        b = np.array(dict_columns[elem]["y"].values)
        W = dict_W[w]
        temp1 = np.dot(A,W) - b
        temp2 = np.square(temp1)
        temp3 = np.mean(temp2)
        dict_r[w] = temp3
    return dict_r

def dirlist(dict_r):
    #计算RMSE  
    dir_list = []
    for elem in dict_r:
        dir_list.append(np.sqrt(dict_r[elem]));
    return dir_list

In [46]:
lin=np.linspace(0.001,0.1,100)
lin

array([0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009,
       0.01 , 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018,
       0.019, 0.02 , 0.021, 0.022, 0.023, 0.024, 0.025, 0.026, 0.027,
       0.028, 0.029, 0.03 , 0.031, 0.032, 0.033, 0.034, 0.035, 0.036,
       0.037, 0.038, 0.039, 0.04 , 0.041, 0.042, 0.043, 0.044, 0.045,
       0.046, 0.047, 0.048, 0.049, 0.05 , 0.051, 0.052, 0.053, 0.054,
       0.055, 0.056, 0.057, 0.058, 0.059, 0.06 , 0.061, 0.062, 0.063,
       0.064, 0.065, 0.066, 0.067, 0.068, 0.069, 0.07 , 0.071, 0.072,
       0.073, 0.074, 0.075, 0.076, 0.077, 0.078, 0.079, 0.08 , 0.081,
       0.082, 0.083, 0.084, 0.085, 0.086, 0.087, 0.088, 0.089, 0.09 ,
       0.091, 0.092, 0.093, 0.094, 0.095, 0.096, 0.097, 0.098, 0.099,
       0.1  ])

In [47]:
dev=100
lin=np.linspace(0.001,1,1000)
for alpha in lin:
    
    dict_W = dictW(alpha)
    
    dict_r = dictr(dict_W)

    dir_list=dirlist(dict_r)
    
    temp=np.mean(dir_list)
                      
    #误差判断
    if(temp>=dev):
        continue
    else:
        dev=temp
        al=alpha

alpha=al
dict_W = dictW(alpha)
dict_r= dictr(dict_W)
dict_r

{"'VINORELBINE', 'DASATINIB'": 41.49185381840285,
 "'CARBOPLATIN', 'DINACICLIB'": 836.9200918909124,
 "'MK-4827', 'DASATINIB'": 110.41403163969139,
 "'CYCLOPHOSPHAMIDE', 'SORAFENIB'": 296.9854713205707,
 "'MITOMYCINE', 'SUNITINIB'": 208.97077544166837,
 "'MITOMYCINE', 'DASATINIB'": 6874.588527137877,
 "'MITOMYCINE', 'ERLOTINIB'": 1213.7015459920924,
 "'TEMOZOLOMIDE', 'SORAFENIB'": 1612.295080287053,
 "'CARBOPLATIN', 'ABT-888'": 2444.5059612251425,
 "'DASATINIB', 'SN-38'": 236.6916924203748,
 "'CYCLOPHOSPHAMIDE', 'DASATINIB'": 986.2732937662014,
 "'BEZ-235', 'OXALIPLATIN'": 1551.001153262388,
 "'ZOLINZA', 'OXALIPLATIN'": 544.3674065535577,
 "'5-FU', 'DASATINIB'": 681.7582126054798,
 "'DASATINIB', 'TOPOTECAN'": 612.728189040412,
 "'ERLOTINIB', 'DASATINIB'": 285.3843401512077}

In [48]:
np.array(list(dict_r.values())).mean()

1158.6298516595646

In [49]:
dict_W

{"'VINORELBINE', 'DASATINIB'": array([0.0965714, 0.0965714, 0.0965714, 0.0965714, 0.0965714]),
 "'CARBOPLATIN', 'DINACICLIB'": array([1.12626244e-13, 3.92609404e-05, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00]),
 "'MK-4827', 'DASATINIB'": array([0.00773089, 0.00773089, 0.00773089, 0.00773089, 1.        ]),
 "'CYCLOPHOSPHAMIDE', 'SORAFENIB'": array([0.00017609, 0.07915715, 0.07915715, 0.07915715, 0.07915715]),
 "'MITOMYCINE', 'SUNITINIB'": array([5.70754173e-06, 2.59313933e-02, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00]),
 "'MITOMYCINE', 'DASATINIB'": array([0.11519547, 0.55745798, 1.        , 1.        , 1.        ]),
 "'MITOMYCINE', 'ERLOTINIB'": array([1.22014535e-10, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00]),
 "'TEMOZOLOMIDE', 'SORAFENIB'": array([1.76092825e-04, 7.91571546e-02, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00]),
 "'CARBOPLATIN', 'ABT-888'": array([1., 1., 1., 1., 1.]),
 "'DASATINIB', 'SN-38'": array([0.0

In [50]:
#5个 446.7268842585183
#10个 451.7241918333837

In [51]:
alpha

0.037000000000000005

In [52]:
# import pickle

# with open('药物对.pkl', 'wb') as file:
#     pickle.dump(dict_W, file)
#     pickle.dump(dict_columns, file)
#     pickle.dump(dict_high_lines, file) 